<a href="https://colab.research.google.com/github/Kedarbhagat/p/blob/main/forcastingSticker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("train.csv")
df= df.drop(columns="id")

In [3]:
df.nunique()

,0
date,2557
country,6
store,3
product,5
num_sold,4037


In [4]:
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day_name()
df["dayofweek"] = df["date"].dt.dayofweek

In [5]:
df["is_weekend"] = np.where(df["day"].isin(["Saturday" ,"Sunday"]),1,0)
df = df.drop(columns="day")

In [6]:
df


,date,country,store,product,num_sold,year,month,dayofweek,is_weekend
0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN,2010,1,4,0
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0,2010,1,4,0
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0,2010,1,4,0
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0,2010,1,4,0
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0,2010,1,4,0
...,...,...,...,...,...,...,...,...,...
230125,2016-12-31,Singapore,Premium Sticker Mart,Holographic Goose,466.0,2016,12,5,1
230126,2016-12-31,Singapore,Premium Sticker Mart,Kaggle,2907.0,2016,12,5,1
230127,2016-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0,2016,12,5,1
230128,2016-12-31,Singapore,Premium Sticker Mart,Kerneler,1242.0,2016,12,5,1


In [7]:
cols = df.columns.tolist()
cols = [col for col in cols if col !="num_sold"] + ["num_sold"]
df = df[cols]

In [8]:
df = df.drop(columns="date")

In [9]:
df.head()

,country,store,product,year,month,dayofweek,is_weekend,num_sold
0,Canada,Discount Stickers,Holographic Goose,2010,1,4,0,NaN
1,Canada,Discount Stickers,Kaggle,2010,1,4,0,973.0
2,Canada,Discount Stickers,Kaggle Tiers,2010,1,4,0,906.0
3,Canada,Discount Stickers,Kerneler,2010,1,4,0,423.0
4,Canada,Discount Stickers,Kerneler Dark Mode,2010,1,4,0,491.0


In [10]:
df= df.drop(columns = ["month","dayofweek"])

In [11]:
df

,country,store,product,year,is_weekend,num_sold
0,Canada,Discount Stickers,Holographic Goose,2010,0,NaN
1,Canada,Discount Stickers,Kaggle,2010,0,973.0
2,Canada,Discount Stickers,Kaggle Tiers,2010,0,906.0
3,Canada,Discount Stickers,Kerneler,2010,0,423.0
4,Canada,Discount Stickers,Kerneler Dark Mode,2010,0,491.0
...,...,...,...,...,...,...
230125,Singapore,Premium Sticker Mart,Holographic Goose,2016,1,466.0
230126,Singapore,Premium Sticker Mart,Kaggle,2016,1,2907.0
230127,Singapore,Premium Sticker Mart,Kaggle Tiers,2016,1,2299.0
230128,Singapore,Premium Sticker Mart,Kerneler,2016,1,1242.0


In [12]:
X_train = df.iloc[: , :-1].values
y_train = df.iloc[: , -1].values

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,1,2])] , remainder="passthrough")
X_train=np.array(ct.fit_transform(X_train))

In [14]:
y_train.reshape(-1,1).flatten()

array([  nan,  973.,  906., ..., 2299., 1242., 1622.])

In [15]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan , strategy="mean")
imputer.fit(y_train[:].reshape(-1,1))
y_train = imputer.transform(y_train.reshape(-1,1)).flatten()

In [16]:
np.isnan(y_train).any()

False

In [17]:
X_train

array([[1.0, 0.0, 0.0, ..., 0.0, 2010, 0],
       [1.0, 0.0, 0.0, ..., 0.0, 2010, 0],
       [1.0, 0.0, 0.0, ..., 0.0, 2010, 0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 2016, 1],
       [0.0, 0.0, 0.0, ..., 0.0, 2016, 1],
       [0.0, 0.0, 0.0, ..., 1.0, 2016, 1]], dtype=object)

In [18]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [19]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

LinearRegression()

In [ ]:
predictions_linearRegression = regressor.predict(X_train)

from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train , y_train)

from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)